In [ ]:
# First, download the provided excerpt of human chromosome 1
# https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

In [3]:
# Second, parse it using the readGenome function we wrote before.
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome



In [48]:
genome = readGenome('chr1.GRCh38.excerpt.fasta')
genome[:100]


'TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAGGTGCATAGGTCAACAATACTTGAGCCTAACTCAGTAGATCCTAAAA'

In [10]:
# Third, adapt the editDistance function we saw in practical (copied below) to answer questions 1 and 2 below. Your function should take arguments p (pattern), 
# t (text) and should return the edit distance of the match between P and T with the fewest edits.

# Hint: In the "A new solution to approximate matching" video we saw that the best approximate match of
# P = GCGTATGC within T = TATTGGCTATACGGTT had 2 edits. You can use this and other small examples to 
# double-check that your function is working.

"""
def editDistance(x, y):
    # x = P (pattern), y= T (reference)
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]
"""

def editDistance(x,y):
    """
    Implement dynamic algorithm to calculate edit distance between a given
    pattern and a given genome
    """
    # x = P (pattern), y= T (reference)
    pattern_length = len(x) + 1
    genome_length = len(y) + 1
    #generate matrix
    matrix = [[0]*genome_length for i in range(pattern_length)]
    # initiate the first column
    for i in range(pattern_length):
        matrix[i][0] = i
    for i in range(1, pattern_length):
        for j in range(1, genome_length):
            dist_hor = matrix[i][j-1] + 1
            dist_vel = matrix[i-1][j] + 1
            if pattern[i-1] == genome[j-1]:
                dist_diag = matrix[i-1][j-1]
            else:
                dist_diag = matrix[i-1][j-1] + 1
            matrix[i][j] = min(dist_hor, dist_vel, dist_diag)
    return matrix[-1][-1]





In [50]:
pattern = 'GCTGATCGATCGTACG'
P = 'GCGTATGC'
T = 'TATTGGCTATACGGTT'
print(editDistance(P,T))


3


In [11]:
from itertools import permutations

class findPatternV3 ():
    """
    This class finds the edit distance of a given pattern in a given genomic 
    sequence and constructs overlap graphs.
    """
    def __init__(self, filename, pattern = False):
        # initiate parameters
        self.pattern = pattern
        self.filename = filename
        
    def readGenome (self):
        """
        read genomic DNA sequence to a string
        """
        genome = ""
        with open (self.filename, "r") as f:
            for line in f:
                # skip header
                if not line[0] == ">":
                    genome += line.rstrip()
            f.close()
        return genome
        
    def readFastq(self):
        """
        read dna sequence and quality base from a fastq sequencing file to lists
        """
        with open (self.filename, "r") as f:
             sequences = []
             qualities = []
             while True:
                 f.readline() # skip name line
                 seq = f.readline().rstrip() # read sequence line
                 f.readline() # skip strand line
                 qual = f.readline().rstrip() # read quality line
                 if len(seq) == 0: #finish read
                     break
                 # add seqence and quality information to list
                 sequences.append(seq)
                 qualities.append(qual)
             f.close()
        return sequences, qualities
        
    def editDistance(self):
        """
        Implement dynamic algorithm to calculate edit distance between a given
        pattern and a given genome
        """
        pattern = self.pattern
        genome = self.readGenome()
        pattern_length = len(pattern) + 1
        genome_length = len(genome) + 1
        #generate matrix
        matrix = [[0]*genome_length for i in range(pattern_length)]
        # initiate the first column
        for i in range(pattern_length):
            matrix[i][0] = i
        for i in range(1, pattern_length):
            for j in range(1, genome_length):
                dist_hor = matrix[i][j-1] + 1
                dist_vel = matrix[i-1][j] + 1
                dist_diag = matrix[i-1][j-1] + 1 if pattern[i-1] != genome[j-1]\
                            else matrix[i-1][j-1]
                matrix[i][j] = min(dist_hor, dist_vel, dist_diag)
        return min(matrix[-1])                
            
    def phraseReads(self, k_mer):
        """
        construct the prefix and suffix of a read to a dictornary with read as
        key and pre-,suffix as values
        """
        reads, _ = self.readFastq()
        reads_dict = {}
        for read in reads:
            for i in range(len(read) - k_mer + 1):
                substring = read[i:i+k_mer]
                if substring not in reads_dict:
                    reads_dict[substring] = set([read])
                else:
                    reads_dict[substring].add(read)
        return reads_dict
    def overlap(self, read1, read2, k_mer):
        """
        find overlaped leftmost offset
        """
        start = 0
        while True:
            start = read1.find(read2[:k_mer], start)
            if start == -1:
                return 0 # without overlap
            if read2.startswith(read1[start:]):
                return len(read1) - start
            start += 1
            
    def overlapGraph(self, k_mer):
        """
        construct graph with key as a read (node) and values as all other 
        reads overlapped with the previous read (node)
        """
        reads_dict = self.phraseReads(k_mer)
        reads, _= self.readFastq()
        graph = {}
        for read1 in reads:
            k_mer_string = read1[len(read1) - k_mer:]
            if k_mer_string in reads_dict:
                edges = set([])
                reads_set = reads_dict[k_mer_string]
                for read2 in reads_set:
                    if read1 != read2: #skip self comparison
                        offset = self.overlap(read1, read2, k_mer)           
                        if offset > 0: # skip non-overlapped pairs
                            edges.add(read2) #add overlapped reads to be values
                            graph[read1] = edges           
        return graph
                        
    def naive_overlap_map(self, k_mer):
        """
        construct graph with key as a pair of reads with overlap and values as
        the leftmost offset of the overlap
        """
        graph = {}
        reads, _ = self.readFastq()
        for read1, read2 in permutations(reads, 2):
            #skip non-overlapped reads
            if read1[len(read1) - k_mer:] in read2:
                offset = self.overlap(read1, read2, k_mer)
                    # check if reads[i] overlapped with reads[j]
                if offset != 0:
                    graph[(read1,read2)] = offset  
        return graph


In [55]:
#Q1: What is the edit distance of the best match between pattern 
#GCTGATCGATCGTACG and the excerpt of human chromosome 1? 
#(Don't consider reverse complements.)
pattern = "GCTGATCGATCGTACG"
filename = "chr1.GRCh38.excerpt.fasta"
patterns = findPatternV3 (filename, pattern)
edit_dist = patterns.editDistance()
print(edit_dist)

3


In [56]:
# Q2: What is the edit distance of the best match between pattern GATTTACCAGATTGAG
#  and the excerpt of human chromosome 1? (Don't consider reverse complements.)
pattern2 = "GATTTACCAGATTGAG"
filename = "chr1.GRCh38.excerpt.fasta"
patterns = findPatternV3 (filename, pattern2)
edit_dist2 = patterns.editDistance()
print(edit_dist2)

2


In [12]:
# Q3: 
# In a practical, we saw a function for finding the longest exact overlap (suffix/prefix match) between two strings. The function is copied below.
# Picture the overlap graph corresponding to the overlaps just calculated. 
# How many edges are in the graph? In other words, how many distinct pairs of reads overlap?


import time
t1 = time.time()
filename = "ERR266411_1.for_asm.fastq"
patterns = findPatternV3 (filename)
k_mer = 30
graph = patterns.naive_overlap_map(k_mer)
t2 = time.time()
print "Running time for naive overlap mapping: %d sec\n"%(t2 - t1)
    
reads = patterns.phraseReads(k_mer)
t3 = time.time() 
print "Running time for phrase reads: %d sec\n"%(t3 - t2)
graph = patterns.overlapGraph(k_mer)
t4 = time.time()
print "Running time for optimized algorithm: %d sec\n"%(t4 - t2)
numberOfNodes = len(graph)
numberOfEdges = sum([len(edges) for edges in graph.values()])
print "Q3: the total edges are %d\n"%numberOfEdges
print "Q4: the total nodes are %d"%numberOfNodes


Running time for naive overlap mapping: 47 sec

Running time for phrase reads: 0 sec

Running time for optimized algorithm: 4 sec

Q3: the total edges are 904746

Q4: the total nodes are 7161
